# Teleoperation
### Establish a connection with the controller:

In [1]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0) # change index if necessary
display(controller)

Controller()

It should say ``Connect gamepad and press any button.`` above until it is connected and receives an input. You should see representations of the 4 axes the joysticks move across and the buttons. If no input received, make sure no other kernels are interfering and/or try reconnecting the controller. This step cannot be combined with linking the controller to the wheels and the controller input display cannot be with the rest of the GUI for some reason.
### Link joysticks to left and right wheels, establish video feed, and enable killswitch:

In [2]:
from jetbot import Robot, Camera, bgr8_to_jpeg, Heartbeat
import traitlets

# link controller to motors

robot = Robot()

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

# establish video feed

image = widgets.Image(format='jpeg', width=300, height=300)
camera = Camera.instance()
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# create killswitch function that ceases the JetBot's functions if disconnected

def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
heartbeat.observe(handle_heartbeat_status, names='status')

Error loading module `ublox_gps`: No module named 'serial'


Left joystick is left wheel and same with right. If JetBot is not moving or there is input delay, try restarting the kernel and running each cell again one by one, waiting for each to finish before moving on to the next.
# Data Collection
### Create directories for data, define functions for image saving, and create GUI:

In [3]:
import os
import ipywidgets as widgets
from IPython.display import display

# Text box for user-defined run suffix (e.g. "run1")
suffix_input = widgets.Text(
    value='',
    placeholder='Enter suffix (e.g. run1)',
    description='Run name:',
    layout=widgets.Layout(width='300px')
)
display(suffix_input)

# Global to store paths initialized only after first save
run_initialized = False
run_dirs = {}

def init_run_dirs():
    global run_initialized, run_dirs
    suffix = suffix_input.value.strip()
    if not suffix:
        print("❌ Please enter a suffix before saving.")
        return False

    base_dir = os.path.join('./data', suffix)
    labels = ['left', 'forward', 'right']
    run_dirs = {}

    for label in labels:
        subfolder = f"{label}_{suffix}"
        full_path = os.path.join(base_dir, subfolder)
        os.makedirs(full_path, exist_ok=True)
        run_dirs[label] = full_path

    run_initialized = True
    print(f"✅ Initialized run folders under: {base_dir}")
    return True

# Save snapshot function
def save_snapshot(label):
    if not run_initialized:
        if not init_run_dirs():
            return  # Abort save if suffix not provided

    dir_path = run_dirs[label]
    filename = f"{label}_{suffix_input.value.strip()}_{len(os.listdir(dir_path))}.jpg"
    image_path = os.path.join(dir_path, filename)

    with open(image_path, 'wb') as f:
        f.write(image.value)

# Save wrapper that updates count and checks init
def save_and_update(label):
    save_snapshot(label)
    count_widgets[label].value = len(os.listdir(run_dirs[label]))

# Button and counter widgets
button_layout = widgets.Layout(width='128px', height='64px')

count_widgets = {
    'left': widgets.IntText(layout=button_layout, value=0),
    'forward': widgets.IntText(layout=button_layout, value=0),
    'right': widgets.IntText(layout=button_layout, value=0),
}

buttons = {
    'left': widgets.Button(description='⬅️ (LB)', layout=button_layout),
    'forward': widgets.Button(description='⬆️ (L2/R2)', layout=button_layout),
    'right': widgets.Button(description='➡️ (RB)', layout=button_layout),
}

# Button click handlers
buttons['left'].on_click(lambda x: save_and_update('left'))
buttons['forward'].on_click(lambda x: save_and_update('forward'))
buttons['right'].on_click(lambda x: save_and_update('right'))

# Controller axis/button binding
def make_button_handler(label):
    def handler(change):
        if change['new'] == 1.0:
            save_and_update(label)
    return handler

controller.buttons[4].observe(make_button_handler('left'), names='value')     # LB
controller.buttons[5].observe(make_button_handler('right'), names='value')    # RB
controller.buttons[6].observe(make_button_handler('forward'), names='value')  # L2
controller.buttons[7].observe(make_button_handler('forward'), names='value')  # R2

# Display GUI
display(widgets.HBox([count_widgets['left'], count_widgets['forward'], count_widgets['right']]))
display(widgets.HBox([buttons['left'], buttons['forward'], buttons['right']]))
display(image)

Text(value='', description='Run name:', layout=Layout(width='300px'), placeholder='Enter suffix (e.g. run1)')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

##### Only in the event the killswitch is activated and the links are severed, use the cell below to reestablish them. If they weren't unlinked, your commands will be multiplied.

In [ ]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)